In [1]:
import pandas as pd
import geopandas as gpd
import altair as alt

In [2]:
%store -r mittel_0
%store -r mittel_4_6
%store -r mittel_8

mittel_0, mittel_4_6, mittel_8

(9.24491, 9.633233571428573, 11.479717857142855)

In [3]:
d3 = pd.read_csv("calc/preis_gemeinde.csv")
d3['strompreis_0'] = mittel_0 - d3.strompreis # 1_marktpreis_bfe.ipynb
d3['strompreis_4_6'] = mittel_4_6 - d3.strompreis
d3['strompreis_8'] = mittel_8 - d3.strompreis
d3[['ID', 'Gemeinde', 'strompreis_0', 'strompreis_4_6', 'strompreis_8']].to_csv("calc/diff_gemeinde.csv", index=False)
d3

,ID,strompreis,Gemeinde,Einwohner,strompreis_0,strompreis_4_6,strompreis_8
0,1,7.551429,Aeugst am Albis,1981,1.693481,2.081805,3.928289
1,2,7.551429,Affoltern am Albis,12303,1.693481,2.081805,3.928289
2,3,7.551429,Bonstetten,5572,1.693481,2.081805,3.928289
3,4,7.551429,Hausen am Albis,3751,1.693481,2.081805,3.928289
4,5,7.551429,Hedingen,3778,1.693481,2.081805,3.928289
...,...,...,...,...,...,...,...
2103,6806,12.214286,Vendlincourt,559,-2.969376,-2.581052,-0.734568
2104,6807,12.214286,Basse-Allaine,1238,-2.969376,-2.581052,-0.734568
2105,6808,12.214286,Clos du Doubs,1263,-2.969376,-2.581052,-0.734568
2106,6809,12.214286,Haute-Ajoie,1085,-2.969376,-2.581052,-0.734568


Geodaten für die Karte lesen:

In [4]:
munis = gpd.read_file('daten/ch-combined.json', layer='municipalities')
munis.id = munis.id.astype(int)

In [5]:
m1 = pd.merge(munis, d3, left_on='id', right_on='ID', how='left')
m1

,id,geometry,ID,strompreis,Gemeinde,Einwohner,strompreis_0,strompreis_4_6,strompreis_8
0,1,"POLYGON ((8.50781 47.28718, 8.51241 47.28423, ...",1.0,7.551429,Aeugst am Albis,1981.0,1.693481,2.081805,3.928289
1,2,"POLYGON ((8.41591 47.29483, 8.43023 47.30202, ...",2.0,7.551429,Affoltern am Albis,12303.0,1.693481,2.081805,3.928289
2,3,"POLYGON ((8.44242 47.31721, 8.45020 47.32474, ...",3.0,7.551429,Bonstetten,5572.0,1.693481,2.081805,3.928289
3,4,"POLYGON ((8.57939 47.22017, 8.57550 47.21840, ...",4.0,7.551429,Hausen am Albis,3751.0,1.693481,2.081805,3.928289
4,5,"POLYGON ((8.43023 47.30202, 8.42952 47.30260, ...",5.0,7.551429,Hedingen,3778.0,1.693481,2.081805,3.928289
...,...,...,...,...,...,...,...,...,...
2129,6808,"POLYGON ((7.03542 47.32981, 7.04656 47.32804, ...",6808.0,12.214286,Clos du Doubs,1263.0,-2.969376,-2.581052,-0.734568
2130,6809,"POLYGON ((6.92196 47.35666, 6.90269 47.36078, ...",6809.0,12.214286,Haute-Ajoie,1085.0,-2.969376,-2.581052,-0.734568
2131,6810,"POLYGON ((7.23690 47.43827, 7.23955 47.43014, ...",6810.0,12.214286,La Baroche,1142.0,-2.969376,-2.581052,-0.734568
2132,6811,"POLYGON ((7.07572 47.48961, 7.09251 47.49621, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def chart(key):
    return alt.Chart(m1).mark_geoshape(stroke='#999', strokeWidth=0.4).encode(
        alt.Color(field=key, type='quantitative').scale(domain=[-3, -2, -1, 0, 1, 2, 3], scheme='redblue'),
        tooltip=[alt.Tooltip(field=key), alt.Tooltip(field='Gemeinde')]
    ).properties(width=900, height=800) 


chart('strompreis_4_6')

alt.Chart(...)

In wie viel Prozent der Gemeinden erhält man mehr für seinen Strom?

Nach Einwohnerzahl:

In [7]:
d3.query('strompreis_0 > 0').Einwohner.sum() /d3.Einwohner.sum()

0.31541231043744467

In [8]:
d3.query('strompreis_4_6 > 0').Einwohner.sum() /d3.Einwohner.sum()

0.3444673053430206

In [9]:
d3.query('strompreis_8 > 0').Einwohner.sum() /d3.Einwohner.sum()

0.6461103568161798

Nach Anzahl Gemeinden:

In [10]:
len(d3.query('strompreis_0 > 0')) / len(d3)

0.2803605313092979

In [11]:
len(d3.query('strompreis_4_6 > 0')) / len(d3)

0.305977229601518

In [12]:
len(d3.query('strompreis_8 > 0')) / len(d3)

0.6740986717267552